In [1]:
import os, sys

parent = os.path.abspath('..')
sys.path.insert(1, parent)

In [2]:
from helper_funcs.preprocessing import total_timeseries, get_covariates
from helper_funcs.inverse import inverse_func
from helper_funcs.error import error_count
from helper_funcs.prediction import model_compare

from darts.utils.model_selection import train_test_split
from darts.dataprocessing.transformers import StaticCovariatesTransformer, Scaler
from pytorch_lightning.callbacks import ModelCheckpoint, RichProgressBar
from darts.models import NLinearModel, TFTModel
from darts import TimeSeries

import pandas as pd 

RANDOM = 101
INPUT_CHUNK = 15
OUTPUT_CHUNK = 3
TEST_SIZE = 0.2
RETRAIN=True
LAST=False
RESET=False
EXP_MA = 15
SAVE = True
EPOCHS = 100

timeseries = total_timeseries(INPUT_CHUNK, market=True, sentiment=True, embeddings=True)
timeseries = StaticCovariatesTransformer().fit_transform(timeseries)

train, val = train_test_split(
    timeseries,
    axis=1,
    test_size=TEST_SIZE,
    input_size=INPUT_CHUNK,
    horizon=OUTPUT_CHUNK,
    vertical_split_type='model-aware'
)

data = [train, val]

target_train, past_train, future_train, target_val, past_val, future_val, target_test, past_test, future_test = get_covariates(
    data,
    target='Close',
    past_covariates=[],
    embeddings=False
    )

scaler_target = Scaler()
scaler_covariates = Scaler()

target_train_scaled = scaler_target.fit_transform(target_train)
target_val_scaled = scaler_target.transform(target_val)
target_test_scaled = scaler_target.transform(target_test)

In [3]:
# names = ['model_nlinear_HLOV', 'model_nlinear_HLOVS1', 'model_nlinear_HLOVS2', 'model_nlinear_HLOVE', 'model_tft_HLOV', 'model_tft_HLOVS1', 'model_tft_HLOVS2', 'model_tft_HLOVE']
names = ['model_tft_HLOVS1']

In [9]:
aapl = model_compare(scaler_target, target_val_scaled, names, 0, (200, 300))
amzn = model_compare(scaler_target, target_val_scaled, names, 1, (200, 300))
goog = model_compare(scaler_target, target_val_scaled, names, 2, (200, 300))
# googl = model_compare(scaler_target, target_val_scaled, names, 3, (0, 300))
msft = model_compare(scaler_target, target_val_scaled, names, 4, (200, 300))
tsla = model_compare(scaler_target, target_val_scaled, names, 5, (200, 300))

           Rank      MAPE       MAE        R2      RMSE       MSE     SMAPE
tft_HLOVS     1  2.838181  1.801166  0.416686  2.170103  4.709346  2.897573
           Rank      MAPE       MAE        R2      RMSE       MSE     SMAPE
tft_HLOVS     1  1.549697  0.815205 -4.529449  1.032105  1.065242  1.551862
           Rank      MAPE       MAE        R2      RMSE       MSE     SMAPE
tft_HLOVS     1  1.345921  0.788653  0.406476  1.012799  1.025761  1.357509
           Rank     MAPE       MAE       R2     RMSE       MSE     SMAPE
tft_HLOVS     1  2.35868  1.695426  0.23406  1.92467  3.704353  2.388104
           Rank      MAPE       MAE        R2      RMSE        MSE     SMAPE
tft_HLOVS     1  8.161342  3.960611  0.141121  4.769636  22.749429  8.568932


In [10]:
aapl = model_compare(scaler_target, target_val_scaled, names, 0, (20, 100))
amzn = model_compare(scaler_target, target_val_scaled, names, 1, (20, 100))
goog = model_compare(scaler_target, target_val_scaled, names, 2, (20, 100))
# googl = model_compare(scaler_target, target_val_scaled, names, 3, (20, 300))
msft = model_compare(scaler_target, target_val_scaled, names, 4, (20, 100))
tsla = model_compare(scaler_target, target_val_scaled, names, 5, (20, 100))

           Rank     MAPE      MAE        R2      RMSE       MSE     SMAPE
tft_HLOVS     1  2.80789  1.29626  0.700494  1.757504  3.088819  2.751571
           Rank      MAPE      MAE        R2      RMSE       MSE     SMAPE
tft_HLOVS     1  1.706704  0.88246  0.709275  1.230603  1.514384  1.689182
           Rank      MAPE      MAE        R2      RMSE       MSE     SMAPE
tft_HLOVS     1  2.028897  1.08516  0.535778  1.443153  2.082689  2.011646
           Rank      MAPE       MAE        R2      RMSE       MSE     SMAPE
tft_HLOVS     1  1.279188  0.766941  0.914257  1.027947  1.056674  1.289071
           Rank       MAPE       MAE       R2      RMSE        MSE      SMAPE
tft_HLOVS     1  13.789393  5.931429 -2.03591  7.658384  58.650851  12.477111


In [8]:
pd.concat({
    'Apple': aapl,
    'Amazon' : amzn,
    'Google': goog,
    'Microsoft': msft,
    'Tesla' : tsla,
    }, axis=0).round(decimals=4)

print(pd.concat({
    'Apple': aapl,
    'Amazon' : amzn,
    'Google': goog,
    'Microsoft': msft,
    'Tesla' : tsla,
    }, axis=0).round(decimals=4).to_latex())

\begin{tabular}{llrrrrrrr}
\toprule
      &           &  Rank &     MAPE &     MAE &      R2 &    RMSE &      MSE &    SMAPE \\
\midrule
Apple & tft\_HLOVS &     1 &   2.8079 &  1.2963 &  0.7005 &  1.7575 &   3.0888 &   2.7516 \\
Amazon & tft\_HLOVS &     1 &   1.7067 &  0.8825 &  0.7093 &  1.2306 &   1.5144 &   1.6892 \\
Google & tft\_HLOVS &     1 &   2.0289 &  1.0852 &  0.5358 &  1.4432 &   2.0827 &   2.0116 \\
Microsoft & tft\_HLOVS &     1 &   1.2792 &  0.7669 &  0.9143 &  1.0279 &   1.0567 &   1.2891 \\
Tesla & tft\_HLOVS &     1 &  13.7894 &  5.9314 & -2.0359 &  7.6584 &  58.6509 &  12.4771 \\
\bottomrule
\end{tabular}



/tmp/ipykernel_37069/868740730.py:15: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  }, axis=0).round(decimals=4).to_latex())
